In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/acl25_gpt4/shuffled_without_module_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/acl25_gpt4/shuffled_without_module_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'A', 'type': 'input port'}, {'id': 1, 'content': 'B', 'type': 'input port'}, {'id': 2, 'content': 'A_greater', 'type': 'output port'}, {'id': 3, 'content': 'A_equal', 'type': 'output port'}, {'id': 4, 'content': 'A_less', 'type': 'output port'}, {'id': 5, 'content': 'higher_cmp', 'type': 'submodule'}, {'id': 6, 'content': 'lower_cmp', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[1, 0, 1, 0, 6, 6, 6], [5, 5, 6, 6, 2, 3, 4]]\n, Design a 16-bit magnitude comparator that evaluates two input numbers, A and B, and outputs three status signals indicating their relative magnitude using smaller 8-bit comparison units.\n\nModule name:\n    magnitude_comparator_16bit\n\nInput ports:\n    A [15:0]: First 16-bit input operand to compare.\n    B [15:0]: Second 16-bit input operand to compare.\nOutput ports:\n    A_greater: 1-bit output indicating if A is greater than B.\n    A_equal: 1-bit output ind

In [6]:
conv[0]['conversations'][1]['value']

'module magnitude_comparator_16bit(\n    input [15:0] A,\n    input [15:0] B,\n    output reg A_greater,\n    output reg A_equal,\n    output reg A_less\n);\n\n    wire A_higher_greater, A_higher_equal, A_higher_less;\n    wire A_lower_greater, A_lower_equal, A_lower_less;\n\n    // Instantiate the higher 8-bit comparator\n    magnitude_comparator_8bit higher_cmp (\n        .A(A[15:8]),\n        .B(B[15:8]),\n        .A_greater(A_higher_greater),\n        .A_equal(A_higher_equal),\n        .A_less(A_higher_less)\n    );\n\n    // Instantiate the lower 8-bit comparator\n    magnitude_comparator_8bit lower_cmp (\n        .A(A[7:0]),\n        .B(B[7:0]),\n        .A_greater(A_lower_greater),\n        .A_equal(A_lower_equal),\n        .A_less(A_lower_less)\n    );\n\n    // Determine the final comparison outputs\n    always @ (*) begin\n        if (A_higher_greater) begin\n            A_greater = 1;\n            A_equal = 0;\n            A_less = 0;\n        end else if (A_higher_less) beg

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'A', 'type': 'input port'}, {'id': 1, 'content': 'B', 'type': 'input port'}, {'id': 2, 'content': 'A_greater', 'type': 'output port'}, {'id': 3, 'content': 'A_equal', 'type': 'output port'}, {'id': 4, 'content': 'A_less', 'type': 'output port'}, {'id': 5, 'content': 'higher_cmp', 'type': 'submodule'}, {'id': 6, 'content': 'lower_cmp', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[1, 0, 1, 0, 6, 6, 6], [5, 5, 6, 6, 2, 3, 4]]\n, Design a 16-bit magnitude comparator that evaluates two input numbers, A and B, and outputs three status signals indicating their relative magnitude using smaller 8-bit comparison units.\n\nModule name:\n    magnitude_comparator_16bit\n\nInput ports:\n    A [15:0]: First 16-bit input operand to compare.\n    B [15:0]: Second 16-bit input operand to compare.\nOutput ports:\n    A_greater: 1-bit output indicating if A is greater than B.\n    A_equal: 1-bit output ind

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
conv[100]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'a', 'type': 'input port'}, {'id': 1, 'content': 'b', 'type': 'input port'}, {'id': 2, 'content': 'Cin', 'type': 'input port'}, {'id': 3, 'content': 'sum', 'type': 'output port'}, {'id': 4, 'content': 'Cout', 'type': 'output port'}, {'id': 5, 'content': 'adder0', 'type': 'submodule'}, {'id': 6, 'content': 'adder1', 'type': 'submodule'}, {'id': 7, 'content': 'adder2', 'type': 'submodule'}, {'id': 8, 'content': 'adder3', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[0, 1, 2, 0, 1, 5, 1, 0, 6, 0, 1, 7, 6, 7, 5, 8, 8], [5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 3, 3, 3, 3, 4]]\n, Design a 32-bit carry-lookahead adder named 'cla_adder_32bit'. This module should utilize smaller carry-lookahead adder blocks, specifically 8-bit blocks named 'cla_adder_8bit', to perform the addition. Each 'cla_adder_8bit' should further use a 'cla_logic_4bit' submodule for carry computation.\n\nModule name:  \n    cla_

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/acl25_gpt4/shuffled_with_module_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [5]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [6]:
len(new_conv)

1326

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148